In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 데이터셋 로드
transform = transforms.Compose([transforms.ToTensor()])
# 훈련 데이터셋 로드
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


# 모델 정의
class Model1(nn.Module): #2 layer no dropout
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.conv2(x), 2))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

class Model2(nn.Module): # 2layer dropout
    def __init__(self):
        super(Model2, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
        
class Model3(nn.Module): # 3 레이어 
    def __init__(self):
        super(Model3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(3 * 3 * 128, 128) 
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 3 * 3 * 128) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Criterion과 Optimizer 정의
criterions = [nn.CrossEntropyLoss(), nn.BCEWithLogitsLoss()]
optimizers = [optim.Adam, optim.SGD, optim.RMSprop]
NB_EPOCH = 10
# 실험 결과를 저장하기 위한 딕셔너리
experiment_losses = {}

# 실험 진행 함수
def train_model(model, criterion, optimizer, epochs=NB_EPOCH):
    model.to(device)  # Move model to the appropriate device
    model.train()
    loss_values = []  
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)  # Move data to the device
            optimizer.zero_grad()
            output = model(data)
            if isinstance(criterion, nn.BCEWithLogitsLoss):
                target = torch.nn.functional.one_hot(target, num_classes=10).float()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        loss_values.append(avg_loss)  

    # 실험 결과 저장
    experiment_label = f'{model.__class__.__name__} with {criterion.__class__.__name__} and {optimizer.__class__.__name__}'
    experiment_losses[experiment_label] = loss_values

# 모델, Criterion, Optimizer 조합
models = [Model1(), Model2(), Model3()]
for model in models:
    for criterion in criterions:
        for opt_class in optimizers:
            optimizer = opt_class(model.parameters(), lr=0.001)
            train_model(model, criterion, optimizer)

# 모든 실험의 손실 그래프 그리기
plt.figure(figsize=(12, 8))
for experiment_label, loss_values in experiment_losses.items():
    plt.plot(range(1, NB_EPOCH+1), loss_values, label=experiment_label)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

KeyboardInterrupt: 

# 최종 선정 모델

In [ ]:
from tqdm import  tqdm
# 테스트 데이터셋 로드
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

train_losses = []
train_accuracies = []

def evaluate_model(model, test_loader, criterion):
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)  # Move data to the device
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

criterion = nn.CrossEntropyLoss()



test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

100%|██████████| 157/157 [00:02<00:00, 66.14it/s]

Test Loss: 0.0362, Test Accuracy: 10.00%
